In [ ]:
import torch 

In [ ]:
# Ensure CUDA support is available 
if torch.cuda.is_available(): 
# Perform operations on GPU 
    device = torch.device("cuda") # ... 
# Clear CUDA memory 
    torch.cuda.empty_cache() 
else: 
    print("CUDA is not available.")

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda") # ... 
# Clear CUDA memory 
torch.cuda.empty_cache() 

In [ ]:
pip install accelerate -U

In [ ]:
!pip install -q accelerate evaluate -U

In [ ]:
!pip install --upgrade transformers

In [ ]:
%pip install --upgrade huggingface_hub

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
import transformers
print(transformers.__version__)

In [ ]:
# from transformers import BertForSequenceClassification, BertTokenizerFast, BertModel\
from transformers import AutoTokenizer, AutoModelForSequenceClassification
#import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from transformers import Trainer, TrainingArguments
import numpy as np
from tqdm.autonotebook import tqdm
from torch import nn

In [ ]:
torch.cuda.is_available()


In [ ]:
device = torch.device("cuda:0")
device

In [ ]:
MAX_LEN = 512

In [ ]:
class BaseDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_len):
        self.data = dataframe
        self.text = dataframe['sentence'].tolist()
        self.targets = None
        if 'labels' in dataframe:
            self.targets = dataframe['labels'].tolist()
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __getitem__(self, index):
        text = str(self.text[index])
        text = ' '.join(text.split())

        inputs = self.tokenizer(
            "query: "+text,
            add_special_tokens=True,
            max_length=self.max_seq_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_tensors="pt",
        )
        text = inputs['input_ids'].squeeze(0)
        attention = inputs['attention_mask'].squeeze(0)
        token_type_ids = inputs["token_type_ids"].squeeze(0)


        if self.targets is not None:
            return {"input_ids": text, "attention_mask": attention, "token_type_ids": token_type_ids, 'labels': torch.tensor(self.targets[index], dtype=torch.long)}
        else:
            return {"input_ids": text, "attention_mask": attention, "token_type_ids": token_type_ids}


    def __len__(self) -> int:
        return len(self.text)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
train_data = pd.read_csv("parsed_reviews.csv")
train_data.head()

In [ ]:
train_data.rename(columns={'sentiment': 'labels'}, inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
le = LabelEncoder()

# Fit the label encoder to the unique labels
unique_labels = ['?', '+', '−']
le.fit(unique_labels)

# Transform the labels using the fitted label encoder
train_data["labels"] = le.transform(train_data["labels"])

# Map the transformed labels to the desired numerical values
label_mapping = {0: 1, 1: 0, 2: 2}
train_data["labels"] = train_data["labels"].map(label_mapping)
train_data

In [ ]:
train_split, val_split = train_test_split(train_data, train_size=0.8, random_state=42, stratify=train_data["labels"])

In [ ]:
def read_model(model_name: str = "DeepPavlov/rubert-base-cased"):
    model = AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(le.classes_))
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

In [ ]:
len(le.classes_)

In [ ]:
model, tokenizer = read_model("intfloat/multilingual-e5-large") 
model.to(device)

In [ ]:
torch.cuda.is_available()

In [ ]:
train_dataset = BaseDataset(train_split, tokenizer, MAX_LEN)
val_dataset = BaseDataset(val_split, tokenizer, MAX_LEN)

In [ ]:
import evaluate
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [ ]:
from transformers import SchedulerType

training_args = TrainingArguments(
    output_dir='./resultse5largefinal2',          # output directory
    num_train_epochs=7,              # total number of training epochs
    # warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,   
    learning_rate = 2e-5,
    # strength of weight decay
    logging_dir='./logsdeepe5largefinal2',            # directory for storing logs
    logging_steps=50,
      evaluation_strategy="epoch",     # perform evaluation each epoch
      #per_device_train_batch_size=32,
      #per_device_eval_batch_size=8,
    auto_find_batch_size=True,
    save_strategy="epoch",
    report_to=None,
   # lr_scheduler_type=SchedulerType.COSINE_WITH_RESTARTS,
    logging_first_step=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()